In [38]:
#import packages
from job_desc import *
import pandas as pd

#Get user input on job description in an array
try:
    inputarr=list()
    usr_in_op,usr_in = '',''

    while usr_in_op != 'X':
        usr_in_op = input("Would you like to enter job description or job URL A: description B. URL X. Exit")
        input_jobs = int(input("How many jobs would you like to enter(integers)? MIN:1 & MAX:10 \n"))

        if input_jobs < 1 or input_jobs > 10:
            print("Number of jobs anticipated is not valid. Please enter an integer between 1 and 10 (both inclusive)")
            usr_in_op = 'X' 

        if usr_in_op == 'A':

            for i in range(input_jobs):
                print("######Job Description number######",i+1)
                inputstr = input("\nEnter job description:")
                #print(inputstr)
                while inputstr == '':
                    print("You are in a time loop. Input job description to get out of it")
                    inputstr = input("\nEnter job description:")
                inputarr.append(inputstr)
            usr_in=usr_in_op
            usr_in_op = 'X'

        elif usr_in_op == 'B':

            for i in range(input_jobs):
                print("######Job URL number######",i+1)
                inputstr = input("\nEnter job URL:")
                #print(inputstr)
                while inputstr == '':
                    print("You are in a time loop. Input job description to get out of it")
                    inputstr = input("\nEnter job description:")
                inputarr.append(inputstr)
                #execute function
            usr_in=usr_in_op
            usr_in_op = 'X'

    if usr_in == 'A':
        df1 = pd.DataFrame(list(zip("", "","","","")), columns = ["Job URL","Job Description","Job Title","Job Location"])
        df1["Job Description"]=inputarr
    elif usr_in == 'B':
        df1 = scrape_job_description(inputarr)

except ValueError:
      print("Error: Number of jobs anticipated is not valid. Please enter an integer between 1 and 10 (both inclusive)")
except Exception:
      print("An unknown error occurred. Please try again with valid inputs.")


        
ignore_df = df1[df1["Job Description"] == "Invalid URL, analysis skipped"]


#Output the invalid job urls and notify user
print ("Invalid jobs will be skipped. Number of jobs skipped in current run are:",len(ignore_df))        
if len(ignore_df) > 0:
    print ("\nFollowing are the details of the jobs skipped:",(ignore_df))          


# This is the dataframe with all valid inputs for downstream assessment
select_df = df1[df1["Job Description"] != "Invalid URL, analysis skipped"]
###Hoping Avishek will merge his calls into this book


Would you like to enter job description or job URL A: description B. URL X. ExitA
How many jobs would you like to enter(integers)? MIN:1 & MAX:10 
2
######Job Description number###### 1

Enter job description:Company Description Dropbox is a leading global collaboration platform that's transforming the way people work together, from the smallest business to the largest enterprise. With more than 700 million registered users across more than 180 countries, our mission is to design a more enlightened way of working. From our headquarters in San Francisco to eight dedicated Studios and a worldwide team of employees who choose where they work best, our Virtual First approach is leading the way into the future of work. Team Description Our Product team advocates for our users and our business, setting the vision for our growing family of products. We use data, research, strategy, and empathy to guide multidisciplinary teams toward a common goal, balancing diverse perspectives and empowering

In [39]:
#If job description input
df1

,Job URL,Job Description,Job Title,Job Location
0,NaN,Company Description Dropbox is a leading globa...,NaN,NaN
1,NaN,simple,NaN,NaN


In [4]:
#If job url input
df1

,Job URL,Job Description,Job Title,Job Location


In [34]:
#selected jobs only
select_df

,Job URL,Job Description,Job Title,Job Location
0,https://www.linkedin.com/jobs/view/3442691887,Company Description Dropbox is a leading globa...,"Dropbox hiring Director of Data Science, New I...","Vancouver, British Columbia, Canada"


In [35]:
#ignored jobs due to error
ignore_df

,Job URL,Job Description,Job Title,Job Location
1,https://www.linkedin.com/jobs/view/3,"Invalid URL, analysis skipped",,


In [36]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
df1['Job Description'][0]

"Company Description Dropbox is a leading global collaboration platform that's transforming the way people work together, from the smallest business to the largest enterprise. With more than 700 million registered users across more than 180 countries, our mission is to design a more enlightened way of working. From our headquarters in San Francisco to eight dedicated Studios and a worldwide team of employees who choose where they work best, our Virtual First approach is leading the way into the future of work. Team Description Our Product team advocates for our users and our business, setting the vision for our growing family of products. We use data, research, strategy, and empathy to guide multidisciplinary teams toward a common goal, balancing diverse perspectives and empowering our teams to do great work. As we scale globally, there’s plenty of space for you to grow alongside us and simplify life for millions of people around the world in team that always focuses on we, not I, and 

In [41]:
df1['Job Description'][0]

"Company Description Dropbox is a leading global collaboration platform that's transforming the way people work together, from the smallest business to the largest enterprise. With more than 700 million registered users across more than 180 countries, our mission is to design a more enlightened way of working. From our headquarters in San Francisco to eight dedicated Studios and a worldwide team of employees who choose where they work best, our Virtual First approach is leading the way into the future of work. Team Description Our Product team advocates for our users and our business, setting the vision for our growing family of products. We use data, research, strategy, and empathy to guide multidisciplinary teams toward a common goal, balancing diverse perspectives and empowering our teams to do great work. As we scale globally, there’s plenty of space for you to grow alongside us and simplify life for millions of people around the world in team that always focuses on we, not I, and 